# Testing data prepared to predict

Strategy: 
- collect an arbitrary data period that contains (begin, end).
- the last value is end. a new_begin is calculated in data_manner.__add_period
- prepare data with build_test(). this automatically removes the last input_window_size from the data to build x, which is the array we will input to predict
- predicts
- get an array of predictions y_hat from new_begin+7 to end
- get the last (end-begin) values from y_hat. this is the forecast from begin to end

In [1]:
configure_json = open('../doc/configure.json', 'r')

import json
#print(json.dumps(json.load(configure_json), indent=4))

In [2]:
import sys
sys.path.append("../src")

import configs_manner

# print("Model infos: \n", configs_manner.model_infos)
# print("\n")
# print("Models path: \n", configs_manner.model_path)
# print("\n")
# print("Data path: \n", configs_manner.data_path)

In [112]:
# import the data_manner.py file. (taking into account that you are in src/ path)
import data_manner
import predictor_manner
from importlib import reload
reload(data_manner)
reload(predictor_manner)

import numpy as np
import datetime

np.set_printoptions(precision=4, suppress=True)

# creating the DataConstructor instance
data_constructor_train = data_manner.DataConstructor()
data_constructor = data_manner.DataConstructor(True)

model_id = "2f36b1b8-c0d1-11ec-9f78-132ad413b391"
# specif code to the remote repository data.
repo = "p971074907"
# country and state acronym splitted by a ":"
path = "brl:am"
# columns (or features) to be extracted from the database, each one splitted by a ":"
feature = "date:newDeaths:"
# start date for the data request.
begin = "2020-04-27"
# finish date for the data request.
end = "2022-05-05"

DATE_FORMAT = "%Y-%m-%d"

# # collect data from the remote repository.
# collected_data_train = data_constructor_train.collect_dataframe(path, repo, feature, begin, end)
# collected_data = data_constructor.collect_dataframe(path, repo, feature, begin, end)

# print("DATA FOR TRAINING", "period:", begin, "-", end)
# # print(collected_data_train)
# # print("\n")
# print("RAW DATA", "period after add_period:", data_constructor.new_first_day.strftime(DATE_FORMAT), "-", end)
# # print("DATA FOR PREDICTING", "period after removing ma buffer:", (data_constructor.new_first_day+datetime.timedelta(days=7)).strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(data_constructor.processed_data_raw)
# print(data_constructor.processed_data_raw.shape[0], "days")
# print("NEW DATA", "period after extrapolate check:", data_constructor.new_first_day.strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
# # print("NEW DATA", "period after check for extrapolate:", data_constructor.new_first_day.strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
# # print("Processed data", data_constructor.processed_data_new.shape[0])
# print(data_constructor.processed_data_new)
# print(data_constructor.processed_data_new.shape[0], "days")

predictor = predictor_manner.PredictorConstructor("2f36b1b8-c0d1-11ec-9f78-132ad413b391", path, repo, feature, begin, end)
y_hat = predictor.predict()

# # print("\n")
# print("DATA X prepared by predictor:", data_constructor.new_first_day.strftime(DATE_FORMAT), data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(predictor.data_X.flatten())
# print(predictor.data_X.flatten().shape)
# # print("\n")
# print("RAW PREDICTION using data_X:", data_constructor.new_first_day.strftime(DATE_FORMAT), data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(predictor.raw_y_hat)
# print(predictor.raw_y_hat.flatten().shape)
# print("OFFSET PREDICTION using data_X:", (data_constructor.new_last_day - datetime.timedelta(days=y_hat.shape[0])).strftime(DATE_FORMAT), data_constructor.new_last_day.strftime(DATE_FORMAT))
# print(y_hat)
# print(y_hat.flatten().shape)

In [113]:
print("RAW: ", predictor.begin_raw, predictor.end_raw)
print("DAYS AVAILABLE: ", predictor.number_of_days_available)
print("DAYS REQUESTED: ", predictor.number_of_days_requested)
print("LEFTOVER: ", predictor.leftover)
print("FORECAST: ", predictor.begin_forecast, predictor.end_forecast)
print("FORECAST SIZE", predictor.raw_y_hat.shape[0])
print(predictor.begin_buffer, predictor.end_buffer, predictor.begin_buffer+predictor.end_buffer)
print("REQUESTED: ", predictor.begin ,predictor.end)

# subtrai da frente
# subtrai de trás

RAW:  2020-03-14 2022-05-03
DAYS AVAILABLE:  781
DAYS REQUESTED:  739
LEFTOVER:  4
FORECAST:  2020-03-25 2022-05-10
FORECAST SIZE 777
33 5 38
REQUESTED:  2020-04-27 2022-05-05


In [114]:
y_hat_weboutput = predictor.predictions_to_weboutput(y_hat)
y_hat_weboutput

[{'date': '2020-04-27', 'prediction': '8.723628'},
 {'date': '2020-04-28', 'prediction': '8.572618'},
 {'date': '2020-04-29', 'prediction': '23.274036'},
 {'date': '2020-04-30', 'prediction': '24.600853'},
 {'date': '2020-05-01', 'prediction': '27.67743'},
 {'date': '2020-05-02', 'prediction': '30.96565'},
 {'date': '2020-05-03', 'prediction': '28.153254'},
 {'date': '2020-05-04', 'prediction': '28.281729'},
 {'date': '2020-05-05', 'prediction': '24.563313'},
 {'date': '2020-05-06', 'prediction': '35.432472'},
 {'date': '2020-05-07', 'prediction': '34.13376'},
 {'date': '2020-05-08', 'prediction': '30.853577'},
 {'date': '2020-05-09', 'prediction': '29.52324'},
 {'date': '2020-05-10', 'prediction': '28.336086'},
 {'date': '2020-05-11', 'prediction': '27.042814'},
 {'date': '2020-05-12', 'prediction': '25.949453'},
 {'date': '2020-05-13', 'prediction': '69.403175'},
 {'date': '2020-05-14', 'prediction': '74.66336'},
 {'date': '2020-05-15', 'prediction': '74.97033'},
 {'date': '2020-05-1

In [101]:
y_hat_weboutput = predictor.predictions_to_weboutput(y_hat)
y_hat_weboutput

[{'date': '2020-04-27', 'prediction': '8.723628'},
 {'date': '2020-04-28', 'prediction': '8.572618'},
 {'date': '2020-04-29', 'prediction': '23.274036'},
 {'date': '2020-04-30', 'prediction': '24.600853'},
 {'date': '2020-05-01', 'prediction': '27.67743'},
 {'date': '2020-05-02', 'prediction': '30.96565'},
 {'date': '2020-05-03', 'prediction': '28.153254'},
 {'date': '2020-05-04', 'prediction': '28.281729'},
 {'date': '2020-05-05', 'prediction': '24.563313'},
 {'date': '2020-05-06', 'prediction': '35.432472'},
 {'date': '2020-05-07', 'prediction': '34.13376'},
 {'date': '2020-05-08', 'prediction': '30.853577'},
 {'date': '2020-05-09', 'prediction': '29.52324'},
 {'date': '2020-05-10', 'prediction': '28.336086'},
 {'date': '2020-05-11', 'prediction': '27.042814'},
 {'date': '2020-05-12', 'prediction': '25.949453'},
 {'date': '2020-05-13', 'prediction': '69.403175'},
 {'date': '2020-05-14', 'prediction': '74.66336'},
 {'date': '2020-05-15', 'prediction': '74.97033'},
 {'date': '2020-05-1

In [62]:
import numpy as np
import matplotlib.pyplot as plt

def uniqueish_color():
    """There're better ways to generate unique colors, but this isn't awful."""
    return plt.cm.gist_ncar(np.random.random())

def plot_weeks(x_predict):
    size = x_predict.x.flatten().shape[0]
    x_axis = list(range(size))

    fig, ax = plt.subplots()

    xy = np.vstack((x_axis, x_predict.x.flatten()))

    for i in range(xy.shape[1]//7):
        if i == 0:
            begin = 0
            end = (i+1)*7
        else:
            begin = (i*7)-1
            end = (i+1)*7
        ax.plot(xy[0][begin:end], xy[1][begin:end], color=uniqueish_color())
    plt.show()

In [12]:
for i in range(100):
    print(i % 7, i // 7)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
0 1
1 1
2 1
3 1
4 1
5 1
6 1
0 2
1 2
2 2
3 2
4 2
5 2
6 2
0 3
1 3
2 3
3 3
4 3
5 3
6 3
0 4
1 4
2 4
3 4
4 4
5 4
6 4
0 5
1 5
2 5
3 5
4 5
5 5
6 5
0 6
1 6
2 6
3 6
4 6
5 6
6 6
0 7
1 7
2 7
3 7
4 7
5 7
6 7
0 8
1 8
2 8
3 8
4 8
5 8
6 8
0 9
1 9
2 9
3 9
4 9
5 9
6 9
0 10
1 10
2 10
3 10
4 10
5 10
6 10
0 11
1 11
2 11
3 11
4 11
5 11
6 11
0 12
1 12
2 12
3 12
4 12
5 12
6 12
0 13
1 13
2 13
3 13
4 13
5 13
6 13
0 14
1 14
